In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
from openai import OpenAI
client = OpenAI()

In [15]:
from drugocr import extract_text
import base64

image_path = "C:\Potenup\Drug-Detection-Chatbot\data\medicine_00806.jpeg"
ocr_texts = extract_text(image_path)    # drugocr.py의 extract_text 함수 사용
ocr_texts_join = ' '.join(ocr_texts)
print(ocr_texts_join)

system_prompt = """
당신은 보수적인 약학 전문가입니다. 약학 전문가로서, 
사용자가 제공한 OCR 텍스트 데이터 확인해서 JSON 형식의 서식을 작성해 주세요.


규칙:
- 제공된 데이터에 없는 항목은 **반드시 "없음"이라고 작성**할 것. 
- 예측이나 추론을 하지 말고, 텍스트를 줄이지 말고 **원본 텍스트를 그대로 작성**할 것. 
- **제품명**: 약의 **제품명**은 별도로 표기된 경우에만 추출하며, '원료약품 및 분량', '성분/함량' 등 **원료나 성분을 설명하는 해당 아래에 있는 텍스트는 제품명으로 간주하지 않습니다**. 
- 제품명이 없으면 "없음"으로 작성하십시오. 
- 성분/함량 ',' 기준으로 나누어 각각 항목으로 처리할 것. 줄바꿈 등으로 잘린 경우는 **반드시 붙여쓰기하여** 원래 단어 그대로 복원할 것. 
- 첨가물/첨가제 ',' 기준으로 나누어 각각 항목으로 처리할 것. 줄바꿈 등으로 잘린 경우는 **반드시 붙여쓰기하여** 원래 단어 그대로 복원할 것. 
- 출력은 반드시 JSON 형식만 출력하고, 그 외 텍스트(설명, 문장)는 절대 포함하지 말 것. 
- **효능/효과**: '효능' 또는 '효과'로 명시된 모든 내용을 **하나도 빠짐없이** 모든 리스트를 작성하십시오.
- OCR 줄바꿈이나 잘림으로 인해 분리된 단어는 반드시 이어 붙여 원래 단어 그대로 복원할 것.
- JSON의 각 key는 반드시 아래 구조를 따를 것

[출력형식] 
{{ 
    '제품명' : "제품의 이름, 없으면 없다고 작성할 것", 
    '성분/함량' : "제품의 각 성분, 유효성분, 함량에 대한 튜플(성분명, 함량)들의 리스트, 없으면 없다고 작성할 것" 
    '첨가물' : "제품의 첨가물/첨가제들의 리스트, 없으면 없다고 작성할 것", 
    '제형/성상' : "제형, 성상, 없으면 없다고 작성할 것", 
    'KPIC/ATC' : 'KPIC/A과C에 대한 리스트, 없으면 없다고 작성할 것", 
    '구분' : "구분, 없으면 없다고 작성할 것" 
    '효능' : "여러가지 효능/효과에 대한 모든 리스트, 없으면 없다고 작성할 것", 
    '용법' : "여러가지 용법에 대한 리스트, 없으면 없다고 작성할 것", 
    '주의사항' : "주의사항 리스트, 없으면 없다고 작성할 것" 
}} 
"""

[원료약품 및 분량] 이 약 1캡슐 중 유효성분:두타스테리드(USP)0.5mg 첨가제(동물유래성분):젤라틴(기원동물:소,사용 부위:가죽) 기타첨가제:농글리세린,부틸히드록시톨루엔,숙 신산젤라틴,폴리옥실40경화피마자유,프로필렌글 리콜모노라우레이트,폴록사머124,D-소르비톨액 [성상] 무색투명한 내용물이 들어있는 미황색의 투명한 타원형 연질캡슐 [효능·효과] 양성 전립선 비대증 증상의 개선,급성 성 요저류 위험성 감소,양성 전립선 비대증과 관련된 수술 필요성 감소, 성인 남성(만18~50세)의 남성형 탈모(androgenetic alopecia)의 치료 [용법·용량] 이 약의 권장용량은 1일 1회 1캡슐(0.5mg)이다. 캡슐 내용물에 노출시 구강 인두점막의 자극을 초래 할 수 있으므로 이 약을 쉽거나 쪼개지 않고 통째로 삼켜 복용해야 한다. 이 약은 식사와 관계없이 복용할 수 있다.신장애 환자 또는 노인 환자에서 이 약의 용량을 조절할 필요는 없다. 간장애 환자에게 이 약을 투여한 자료가 없기 때문에 간장애 환자에서의 이 약의 권장용량은 확립되어 있지 않다. [사용상의 주의사항] 1.경고1)여성에게 노출시 남자 태아에 미치는 위험성 이 약은 피부를 통 통해서 서 흡수된다.따라서 이 약의 흡수 가능성과 남자 태아에게 미치는 태자 기형의 위험 가능성 때문에 임신했거나 임신기 가능성이 있는 여성이 이 약을 취급해서는 안든 된다. 또, 여성은 이 약을 취급할 때마다 주의해야 하고 누출되는 캡슐 과의 접촉을 피해야 한다. 만약 캡슐이 새어 이 약과 접촉한 경우에는 접촉부위를 즉시 물과 비누로 세척해야 한다. 이하 첨부문서 참조 [저장방법] 밀폐용기,30°C이하 보관 [제조의뢰자](주)제뉴원사이언스 세종특별자치시 I|전의면 산단길 245 [제조자](주)유유제약 충북 제천시 바이오밸리 1로 94


In [22]:
# print(system_prompt)
print(type(ocr_texts_join))

<class 'str'>


In [25]:
def chat(system_prompt, user):
    response = client.chat.completions.create(
        model="gpt-4.1-nano",
        messages = [
    
        {
            "role": "system",
            "content":[{"type": "text", "text": system_prompt},]
        },
        {
            "role": user,
            "content": [
                {"type": "text", "text": ocr_texts_join},  # "text": ocr_texts 와 "text" : f"{ocr_texts}" 같은 표현
            ]
        }
    ],
    temperature=0
    )

    return response


In [26]:
answer = chat(system_prompt, "user")
print(answer)

ChatCompletion(id='chatcmpl-CItCGVqiWvBsHh3SzTxIHt3Yr4DjI', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{\n  "제품명": "없음",\n  "성분/함량": [\n    ["두타스테리드(USP)", "0.5mg"]\n  ],\n  "첨가물": [\n    "젤라틴(기원동물:소,사용 부위:가죽)",\n    "농글리세린",\n    "부틸히드록시톨루엔",\n    "숙 신산젤라틴",\n    "폴리옥실40경화피마자유",\n    "프로필렌글리콜모노라우레이트",\n    "폴록사머124",\n    "D-소르비톨액"\n  ],\n  "제형/성상": "무색투명한 내용물이 들어있는 미황색의 투명한 타원형 연질캡슐",\n  "KPIC/ATC": "없음",\n  "구분": "없음",\n  "효능": [\n    "양성 전립선 비대증 증상의 개선",\n    "급성 성 요저류 위험성 감소",\n    "양성 전립선 비대증과 관련된 수술 필요성 감소",\n    "성인 남성(만18~50세)의 남성형 탈모(androgenetic alopecia)의 치료"\n  ],\n  "용법": [\n    "이 약의 권장용량은 1일 1회 1캡슐(0.5mg)이다",\n    "캡슐 내용물에 노출시 구강 인두점막의 자극을 초래 할 수 있으므로 이 약을 쉽거나 쪼개지 않고 통째로 삼켜 복용해야 한다",\n    "이 약은 식사와 관계없이 복용할 수 있다",\n    "신장애 환자 또는 노인 환자에서 이 약의 용량을 조절할 필요는 없다",\n    "간장애 환자에게 이 약을 투여한 자료가 없기 때문에 간장애 환자에서의 이 약의 권장용량은 확립되어 있지 않다"\n  ],\n  "주의사항": [\n    "여성에게 노출시 남자 태아에 미치는 위험성 이 약은 피부를 통 통해서 서 흡수된다",\n    "따라서 이 약의 

In [ ]:
import json 
print(answer.choices[0].message.content) # 문자열
print(json.loads(answer.choices[0].message.content)) # 딕셔너리

{'제품명': '없음',
 '성분/함량': [['두타스테리드(USP)', '0.5mg']],
 '첨가물': ['젤라틴(기원동물:소,사용 부위:가죽)',
  '농글리세린',
  '부틸히드록시톨루엔',
  '숙 신산젤라틴',
  '폴리옥실40경화피마자유',
  '프로필렌글리콜모노라우레이트',
  '폴록사머124',
  'D-소르비톨액'],
 '제형/성상': '무색투명한 내용물이 들어있는 미황색의 투명한 타원형 연질캡슐',
 'KPIC/ATC': '없음',
 '구분': '없음',
 '효능': ['양성 전립선 비대증 증상의 개선',
  '급성 성 요저류 위험성 감소',
  '양성 전립선 비대증과 관련된 수술 필요성 감소',
  '성인 남성(만18~50세)의 남성형 탈모(androgenetic alopecia)의 치료'],
 '용법': ['이 약의 권장용량은 1일 1회 1캡슐(0.5mg)이다',
  '캡슐 내용물에 노출시 구강 인두점막의 자극을 초래 할 수 있으므로 이 약을 쉽거나 쪼개지 않고 통째로 삼켜 복용해야 한다',
  '이 약은 식사와 관계없이 복용할 수 있다',
  '신장애 환자 또는 노인 환자에서 이 약의 용량을 조절할 필요는 없다',
  '간장애 환자에게 이 약을 투여한 자료가 없기 때문에 간장애 환자에서의 이 약의 권장용량은 확립되어 있지 않다'],
 '주의사항': ['여성에게 노출시 남자 태아에 미치는 위험성 이 약은 피부를 통 통해서 서 흡수된다',
  '따라서 이 약의 흡수 가능성과 남자 태아에게 미치는 태자 기형의 위험 가능성 때문에 임신했거나 임신기 가능성이 있는 여성이 이 약을 취급해서는 안든 된다',
  '또, 여성은 이 약을 취급할 때마다 주의해야 하고 누출되는 캡슐 과의 접촉을 피해야 한다',
  '만약 캡슐이 새어 이 약과 접촉한 경우에는 접촉부위를 즉시 물과 비누로 세척해야 한다']}

In [33]:
from dotenv import load_dotenv
load_dotenv()
from openai import OpenAI
client = OpenAI()

from drugocr import extract_text
import base64

image_path = "C:\Potenup\Drug-Detection-Chatbot\data\medicine_00806.jpeg"
ocr_texts = extract_text(image_path)    # drugocr.py의 extract_text 함수 사용
ocr_texts_join = ' '.join(ocr_texts)
# print(ocr_texts_join)

system_prompt = """
당신은 보수적인 약학 전문가입니다. 약학 전문가로서, 
사용자가 제공한 OCR 텍스트 데이터 확인해서 JSON 형식의 서식을 작성해 주세요.


규칙:
- 제공된 데이터에 없는 항목은 **반드시 "없음"이라고 작성**할 것. 
- 예측이나 추론을 하지 말고, 텍스트를 줄이지 말고 **원본 텍스트를 그대로 작성**할 것. 
- **제품명**: 약의 **제품명**은 별도로 표기된 경우에만 추출하며, '원료약품 및 분량', '성분/함량' 등 **원료나 성분을 설명하는 해당 아래에 있는 텍스트는 제품명으로 간주하지 않습니다**. 
- 제품명이 없으면 "없음"으로 작성하십시오. 
- 성분/함량 ',' 기준으로 나누어 각각 항목으로 처리할 것. 줄바꿈 등으로 잘린 경우는 **반드시 붙여쓰기하여** 원래 단어 그대로 복원할 것. 
- 첨가물/첨가제 ',' 기준으로 나누어 각각 항목으로 처리할 것. 줄바꿈 등으로 잘린 경우는 **반드시 붙여쓰기하여** 원래 단어 그대로 복원할 것. 
- 출력은 반드시 JSON 형식만 출력하고, 그 외 텍스트(설명, 문장)는 절대 포함하지 말 것. 
- **효능/효과**: '효능' 또는 '효과'로 명시된 모든 내용을 **하나도 빠짐없이** 모든 리스트를 작성하십시오.
- OCR 줄바꿈이나 잘림으로 인해 분리된 단어는 반드시 이어 붙여 원래 단어 그대로 복원할 것.
- JSON의 각 key는 반드시 아래 구조를 따를 것

[출력형식] 
{{ 
    '제품명' : "제품의 이름, 없으면 없다고 작성할 것", 
    '성분/함량' : "제품의 각 성분, 유효성분, 함량에 대한 튜플(성분명, 함량)들의 리스트, 없으면 없다고 작성할 것" 
    '첨가물' : "제품의 첨가물/첨가제들의 리스트, 없으면 없다고 작성할 것", 
    '제형/성상' : "제형, 성상, 없으면 없다고 작성할 것", 
    'KPIC/ATC' : 'KPIC/A과C에 대한 리스트, 없으면 없다고 작성할 것", 
    '구분' : "구분, 없으면 없다고 작성할 것" 
    '효능' : "여러가지 효능/효과에 대한 모든 리스트, 없으면 없다고 작성할 것", 
    '용법' : "여러가지 용법에 대한 리스트, 없으면 없다고 작성할 것", 
    '주의사항' : "주의사항 리스트, 없으면 없다고 작성할 것" 
}} 
"""
def chat(system_prompt, user):
    response = client.chat.completions.create(
        model="gpt-4.1-nano",
        messages = [
    
        {
            "role": "system",
            "content":[{"type": "text", "text": system_prompt},]
        },
        {
            "role": user,
            "content": [
                {"type": "text", "text": ocr_texts_join},  # "text": ocr_texts 와 "text" : f"{ocr_texts}" 같은 표현
            ]
        }
    ],
    temperature=0
    )

    return response

answer = chat(system_prompt, "user")

import json 
print(json.loads(answer.choices[0].message.content))

{'제품명': '없음', '성분/함량': [['두타스테리드(USP)', '0.5mg']], '첨가물': ['젤라틴(기원동물:소,사용 부위:가죽)', '농글리세린', '부틸히드록시톨루엔', '숙 신산젤라틴', '폴리옥실40경화피마자유', '프로필렌글리콜모노라우레이트', '폴록사머124', 'D-소르비톨액'], '제형/성상': '무색투명한 내용물이 들어있는 미황색의 투명한 타원형 연질캡슐', 'KPIC/ATC': '없음', '구분': '없음', '효능': ['양성 전립선 비대증 증상의 개선', '급성 성 요저류 위험성 감소', '양성 전립선 비대증과 관련된 수술 필요성 감소', '성인 남성(만18~50세)의 남성형 탈모(androgenetic alopecia)의 치료'], '용법': ['이 약의 권장용량은 1일 1회 1캡슐(0.5mg)이다', '캡슐 내용물에 노출시 구강 인두점막의 자극을 초래 할 수 있으므로 이 약을 쉽거나 쪼개지 않고 통째로 삼켜 복용해야 한다', '이 약은 식사와 관계없이 복용할 수 있다', '신장애 환자 또는 노인 환자에서 이 약의 용량을 조절할 필요는 없다', '간장애 환자에게 이 약을 투여한 자료가 없기 때문에 간장애 환자에서의 이 약의 권장용량은 확립되어 있지 않다'], '주의사항': ['여성에게 노출시 남자 태아에 미치는 위험성 이 약은 피부를 통 통해서 서 흡수된다', '따라서 이 약의 흡수 가능성과 남자 태아에게 미치는 태자 기형의 위험 가능성 때문에 임신했거나 임신기 가능성이 있는 여성이 이 약을 취급해서는 안든 된다', '또, 여성은 이 약을 취급할 때마다 주의해야 하고 누출되는 캡슐 과의 접촉을 피해야 한다', '만약 캡슐이 새어 이 약과 접촉한 경우에는 접촉부위를 즉시 물과 비누로 세척해야 한다']}
